In [4]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [5]:
import numpy as np
import pandas as pd

In [6]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [7]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [8]:
##################################################
# Databse Setup
##################################################
engine = create_engine("sqlite:///hawaii.sqlite")

In [9]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [10]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [11]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [15]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
#Starting from the last data point in the database. 
prev_year = dt.date(2017, 8, 23)
# Calculate the date one year from the last date in data set.
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year).all()
print(results)
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=['date','precipitation'])
df.set_index(df['date'], inplace=True)
# Sort the dataframe by date
df = df.sort_index()
print(df.to_string(index=False))
# Use Pandas Plotting with Matplotlib to plot the data
df.plot()
plt.xticks(rotation=90)
plt.legend(loc="upper right")

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# How many stations are available in this dataset?
session.query(func.count(Station.station)).all()

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.
session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results = session.query(Measurement.tobs).\
filter(Measurement.station == 'USC00519281').\
filter(Measurement.date >= prev_year).all()
# print(results)
df = pd.DataFrame(results, columns=['tobs'])
print(df)
# Plot temperature
df.plot.hist(bins=5)
plt.tight_layout()

In [41]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps():
    
    start_date_entry = input('Enter a start date in YYYY-MM-DD format ')
    start_year, start_month, start_day = map(int, start_date_entry.split('-'))
    start = dt.date(start_year, start_month, start_day)
    
    end_date_entry = input('Enter an end date in YYYY-MM-DD format ')
    year, month, day = map(int, end_date_entry.split('-'))
    end = dt.date(year, month, day)
   
    
    sel = [func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)]
    
    results = session.query(*sel).\
        filter(Measurement.date >= start).\
            filter(Measurement.date <= end).all()
    
    temps = list(np.ravel(results)) 
    
    return (temps)

In [42]:
calc_temps()

Enter a start date in YYYY-MM-DD format 2017-01-01
Enter an end date in YYYY-MM-DD format 2017-01-30


[60.0, 81.0, 69.7445652173913]

# Challenge

In [45]:
def monthly_stats():
    
    start_date_entry = input('Enter 2017 start date in YYYY-MM-DD format ')
    start_year, start_month, start_day = map(int, start_date_entry.split('-'))
    start = dt.date(start_year, start_month, start_day)
    
    end_date_entry = input('Enter 2017 end date in YYYY-MM-DD format ')
    year, month, day = map(int, end_date_entry.split('-'))
    end = dt.date(year, month, day)
   
    
    # start and end date
    start = start
    end = end

    start_2016 = start - dt.timedelta(days=365)
    end_2016 = end -dt.timedelta(days=365)

    start_2015 = start_2016 - dt.timedelta(days=366)
    end_2015 = end_2016 -dt.timedelta(days=366)

    start_2014 = start_2015 - dt.timedelta(days=365)
    end_2014 = end_2015 -dt.timedelta(days=365)

    start_2013 = start_2014 - dt.timedelta(days=365)
    end_2013 = end_2014 -dt.timedelta(days=365)

    start_2012 = start_2013 - dt.timedelta(days=365)
    end_2012 = end_2013 -dt.timedelta(days=365)

    start_2011 = start_2012 - dt.timedelta(days=366)
    end_2011 = end_2012 -dt.timedelta(days=366)

    start_2010 = start_2011 - dt.timedelta(days=365)
    end_2010 = end_2011 -dt.timedelta(days=365)


    # Perform a query to retrieve the data and precipitation scores
    results = session.query(Measurement.date, Measurement.prcp, Measurement.tobs).\
              filter(((Measurement.date >= start) & (Measurement.date <= end)) |\
                     ((Measurement.date >= start_2016) & (Measurement.date <= end_2016)) |\
                     ((Measurement.date >= start_2015) & (Measurement.date <= end_2015)) |\
                     ((Measurement.date >= start_2014) & (Measurement.date <= end_2014)) |\
                     ((Measurement.date >= start_2013) & (Measurement.date <= end_2013)) |\
                     ((Measurement.date >= start_2012) & (Measurement.date <= end_2012)) |\
                     ((Measurement.date >= start_2011) & (Measurement.date <= end_2011)) |\
                     ((Measurement.date >= start_2010) & (Measurement.date <= end_2010))).\
              all()

    #print(results)

    # Save the query results as a Pandas DataFrame and set the index to the date column
    df = pd.DataFrame(results, columns=['date','precipitation','temperature'])
    df
    df.set_index(df['date'], inplace=True)
    # Sort the dataframe by date
    df = df.sort_index()
    df_stats = df.describe()

    return(df_stats)

In [46]:
monthly_stats()

Enter 2017 start date in YYYY-MM-DD format 2017-06-01
Enter 2017 end date in YYYY-MM-DD format 2017-06-30


,precipitation,temperature
count,1574.000000,1700.000000
mean,0.136360,74.944118
std,0.335731,3.257417
min,0.000000,64.000000
25%,0.000000,73.000000
50%,0.020000,75.000000
75%,0.120000,77.000000
max,4.430000,85.000000


In [47]:
monthly_stats()

Enter 2017 start date in YYYY-MM-DD format 2017-12-01
Enter 2017 end date in YYYY-MM-DD format 2017-12-31


,precipitation,temperature
count,1405.000000,1517.000000
mean,0.216819,71.041529
std,0.541399,3.745920
min,0.000000,56.000000
25%,0.000000,69.000000
50%,0.030000,71.000000
75%,0.150000,74.000000
max,6.420000,83.000000
